#Match timestamps in color and depth images



In [1]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Read Color image dataframe and depth info dataframe (since depth info is too large to fit)
df_color = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/D435I-color-image_raw.csv")
#df_color = df_color.drop(columns=['data'])

In [7]:
df_depth_info = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/D435I-depth-camera_info.csv")
df_depth_info

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,height,width,distortion_model,D_0,D_1,D_2,D_3,D_4,K_0,K_1,K_2,K_3,K_4,K_5,K_6,K_7,K_8,R_0,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,P_0,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10,P_11,binning_x,binning_y,roi.x_offset,roi.y_offset,roi.height,roi.width,roi.do_rectify
0,1.607028e+09,0,1607028351,497767075,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
1,1.607028e+09,1,1607028351,603806696,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
2,1.607028e+09,2,1607028351,695615744,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
3,1.607028e+09,3,1607028351,808113321,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
4,1.607028e+09,4,1607028352,381082589,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4028,1.607029e+09,4028,1607028669,339361999,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
4029,1.607029e+09,4029,1607028669,404573157,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
4030,1.607029e+09,4030,1607028669,560991885,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False
4031,1.607029e+09,4031,1607028669,665631851,D435I_depth_optical_frame,480,640,plumb_bob,0.0,0.0,0.0,0.0,0.0,382.442932,0.0,316.994629,0.0,382.442932,236.173904,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,382.442932,0.0,316.994629,0.0,0.0,382.442932,236.173904,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,False


In [11]:
df_t265_odom = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/T265-odom-sample.csv")
df_t265_odom

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,child_frame_id,pose.pose.position.x,pose.pose.position.y,pose.pose.position.z,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z,pose.pose.orientation.w,pose.covariance,twist.twist.linear.x,twist.twist.linear.y,twist.twist.linear.z,twist.twist.angular.x,twist.twist.angular.y,twist.twist.angular.z,twist.covariance
0,1.607028e+09,0,1607028350,739261627,T265_odom_frame,T265_pose_frame,0.000204,-0.000154,-0.000129,-0.018735,-0.003168,-0.000050,0.999820,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...",-0.002557,-0.002345,-0.003509,0.001124,-0.002322,0.006932,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."
1,1.607028e+09,1,1607028350,744263649,T265_odom_frame,T265_pose_frame,0.000205,-0.000170,-0.000145,-0.018735,-0.003173,-0.000031,0.999819,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...",-0.001598,-0.002723,-0.003586,0.000059,0.006076,0.001738,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."
2,1.607028e+09,2,1607028351,244425774,T265_odom_frame,T265_pose_frame,-0.000088,-0.000032,0.000235,-0.020210,0.000278,0.000108,0.999796,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...",-0.000943,-0.000585,-0.001606,-0.002227,-0.001805,0.001947,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."
3,1.607028e+09,3,1607028351,249426842,T265_odom_frame,T265_pose_frame,-0.000094,-0.000035,0.000227,-0.020213,0.000271,0.000114,0.999796,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...",-0.000951,-0.000605,-0.001730,-0.000110,0.000229,-0.000096,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."
4,1.607028e+09,4,1607028351,254428625,T265_odom_frame,T265_pose_frame,-0.000103,-0.000039,0.000216,-0.020212,0.000275,0.000112,0.999796,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...",-0.001496,-0.000630,-0.002110,-0.000086,-0.000729,0.001948,"(0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717,1.607029e+09,2717,1607028668,690917730,T265_odom_frame,T265_pose_frame,-5.118776,-0.104737,-0.145842,-0.022299,-0.015367,-0.012080,0.999560,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0...",-0.030858,-0.006159,0.003623,-0.000117,0.000545,-0.002505,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0..."
2718,1.607029e+09,2718,1607028668,981035948,T265_odom_frame,T265_pose_frame,-5.119008,-0.102724,-0.148364,-0.022325,-0.015433,-0.012087,0.999559,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0...",-0.034558,0.001269,-0.001683,-0.003246,-0.001693,0.001840,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0..."
2719,1.607029e+09,2719,1607028668,986037731,T265_odom_frame,T265_pose_frame,-5.119182,-0.102713,-0.148365,-0.022340,-0.015444,-0.012091,0.999558,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0...",-0.035247,0.001218,-0.000282,0.000757,-0.002891,-0.001510,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0..."
2720,1.607029e+09,2720,1607028669,221133471,T265_odom_frame,T265_pose_frame,-5.117958,-0.101792,-0.147510,-0.022357,-0.015434,-0.012142,0.999557,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0...",-0.031333,0.002385,0.003872,0.003064,-0.001852,-0.002406,"(0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0..."


In [ ]:
df_t265_gyro = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/T265-gyro-sample.csv")
df_t265_gyro

In [ ]:
df_t265_accel = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/T265-accel-sample.csv")
df_t265_accel

In [ ]:
df_fisheye2 = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/T265-fisheye2-image_raw.csv")
df_fisheye2

In [ ]:
df_fisheye1 = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/T265-fisheye1-image_raw.csv")
df_fisheye1

In [ ]:
df_d435i_accel = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/D435I-accel-sample.csv")
df_d435i_accel

In [ ]:
df_d435i_gyro = pd.read_csv("/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/D435I-gyro-sample.csv")
df_d435i_gyro

In [5]:
#df_color = df_color.drop(columns=['data'])
df_color.to_csv('/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/D435I-color-image_raw.csv')


In [ ]:
# Remove data column from color dataframe to make it lean
#df_color = df_color.drop(columns=['data'])


# Add a new column to store header.seq corresponding to depth frame
df_color['depth_frame_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,3):#len(df_color)):
    rows = df_depth_info.loc[(df_depth_info['header.stamp.secs'] == df_color['header.stamp.secs'][i])
                          | (df_depth_info['header.stamp.secs'] == df_color['header.stamp.secs'][i]-1)
                          | (df_depth_info['header.stamp.secs'] == df_color['header.stamp.secs'][i]+1) ]
    
    print(rows)
    
    if not rows.empty:
       first_index = rows.first_valid_index()
       min_nanosec_diff = abs(rows['header.stamp.secs'][first_index]+rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.secs'][i]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.secs'][j]+rows['header.stamp.nsecs'][j]-df_color['header.stamp.secs'][i]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['depth_frame_index'][i]=min_index
      #  print('final min index',min_index)
      # print(min_nanosec_diff)
# print(rows)
# print(min_nanosec_diff)
# print(min_index)
df_color
   
   # Populate new col with row 

In [44]:
# Add a new column to store header.seq corresponding to depth frame
df_color['depth_frame_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,5):#len(df_color)): 
    rows = df_depth_info.loc[(df_depth_info['header.stamp.secs'] == df_color['header.stamp.secs'][i]-1)
                          | (df_depth_info['header.stamp.secs'] == df_color['header.stamp.secs'][i])
                          | (df_depth_info['header.stamp.secs'] == df_color['header.stamp.secs'][i]+1) ]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       if (rows['header.stamp.secs'][first_index] == df_color['header.stamp.secs'][i] -1):
          min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-1000000000-df_color['header.stamp.nsecs'][i])
       elif (rows['header.stamp.secs'][first_index] == df_color['header.stamp.secs'][i] ):
          min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       else:
          min_nanosec_diff = abs(1000000000+rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
      #  print('mindiff',min_nanosec_diff) 
       min_index = rows['header.seq'][first_index] 
       print('first min index',min_index) 
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             if (rows['header.stamp.secs'][j]== df_color['header.stamp.secs'][i]-1 ):
                new_diff = abs(rows['header.stamp.nsecs'][j]-1000000000-df_color['header.stamp.nsecs'][i])
                # print('here1')
                # print('newdiff mindiff',new_diff,min_nanosec_diff)
             elif (rows['header.stamp.secs'][j] == df_color['header.stamp.secs'][i] ):
                new_diff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             else:
                new_diff = abs(1000000000+rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])

             if (newdiff < min_nanosec_diff):
                  min_nanosec_diff =  newdiff
                  min_index =  rows['header.seq'][j]
                  print('new min index', min_index)
            
             #print(newdiff) 
             
       df_color['depth_frame_index'][i]=min_index
      #  print('final min index', min_index)
      #  print('--------------------------')
df_color

first min index 0
first min index 0
new min index 1
first min index 0
new min index 1
first min index 0
new min index 1
first min index 0
new min index 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to b

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,height,width,encoding,is_bigendian,step,depth_frame_index,odom_index
0,1.607028e+09,0,1607028351,497767075,D435I_color_optical_frame,480,640,rgb8,0,1920,0,1
1,1.607028e+09,1,1607028351,603806696,D435I_color_optical_frame,480,640,rgb8,0,1920,1,1
2,1.607028e+09,2,1607028351,695615744,D435I_color_optical_frame,480,640,rgb8,0,1920,1,1
3,1.607028e+09,3,1607028351,808113321,D435I_color_optical_frame,480,640,rgb8,0,1920,1,1
4,1.607028e+09,4,1607028351,960551103,D435I_color_optical_frame,480,640,rgb8,0,1920,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1.607029e+09,1460,1607028668,704241542,D435I_color_optical_frame,480,640,rgb8,0,1920,0,2694
1461,1.607029e+09,1461,1607028668,930797739,D435I_color_optical_frame,480,640,rgb8,0,1920,0,2694
1462,1.607029e+09,1462,1607028669,235407810,D435I_color_optical_frame,480,640,rgb8,0,1920,0,2714
1463,1.607029e+09,1463,1607028669,540289390,D435I_color_optical_frame,480,640,rgb8,0,1920,0,2714


In [36]:
mindiff=197848669
newdiff =0

if (newdiff < mindiff):
  print('we have a new diff')


we have a new diff


In [19]:
# Add a new column to store header.seq corresponding to depth frame
df_color['odom_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
   # rows = df_t265_odom.loc[df_t265_odom['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    rows = df_t265_odom.loc[(df_t265_odom['header.stamp.secs'] == df_color['header.stamp.secs'][i]-1)
                          | (df_t265_odom['header.stamp.secs'] == df_color['header.stamp.secs'][i])
                          | (df_t265_odom['header.stamp.secs'] == df_color['header.stamp.secs'][i]+1) ]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       if (rows['header.stamp.secs'][first_index] == df_color['header.stamp.secs'][i]-1 ):
          min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-1000000000-df_color['header.stamp.nsecs'][i])
       elif (rows['header.stamp.secs'][first_index] == df_color['header.stamp.secs'][i] ):
          min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       elif (rows['header.stamp.secs'][first_index] == df_color['header.stamp.secs'][i]+1 ):
          min_nanosec_diff = abs(1000000000+rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       else: 
         pass  
       min_index = rows['header.seq'][first_index]  
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             if (rows['header.stamp.secs'][j] == df_color['header.stamp.secs'][i]-1 ):
                new_diff = abs(rows['header.stamp.nsecs'][j]-1000000000-df_color['header.stamp.nsecs'][i])
             elif (rows['header.stamp.secs'][j] == df_color['header.stamp.secs'][i] ):
                new_diff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             elif (rows['header.stamp.secs'][j] == df_color['header.stamp.secs'][i]+1 ):
                new_diff = abs(1000000000+rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             else: 
               pass  
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['odom_index'][i]=min_index

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

In [ ]:
# Add a new column to store header.seq corresponding to depth frame
df_color['gyro_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
    rows = df_t265_gyro.loc[df_t265_gyro['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['gyro_index'][i]=min_index

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

In [ ]:
# Add a new column to store header.seq corresponding to depth frame
df_color['accel_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
    rows = df_t265_accel.loc[df_t265_accel['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['accel_index'][i]=min_index

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

In [ ]:
# Add a new column to store header.seq corresponding to depth frame
df_color['fisheye2_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
    rows = df_fisheye2.loc[df_fisheye2['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['fisheye2_index'][i]=min_index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to b

In [ ]:
# Add a new column to store header.seq corresponding to depth frame
df_color['fisheye1_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
    rows = df_fisheye1.loc[df_fisheye1['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['fisheye1_index'][i]=min_index

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

In [ ]:
# Add a new column to store header.seq corresponding to depth frame
df_color['d435i_accel_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
    rows = df_d435i_accel.loc[df_d435i_accel['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['d435i_accel_index'][i]=min_index

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

In [ ]:
# Add a new column to store header.seq corresponding to depth frame
df_color['d435i_gyro_index']=0
# For each row in color dataframe, find the secs time stamp match
for i in range(0,len(df_color)): 
    rows = df_d435i_gyro.loc[df_d435i_gyro['header.stamp.secs'] == df_color['header.stamp.secs'][i]]
    #print(len(rows))
    if not rows.empty:
       #print(rows)
       first_index = rows.first_valid_index()
       #print(rows['header.stamp.nsecs'][0])
       #df_color['header.stamp.nsecs'][i]
       min_nanosec_diff = abs(rows['header.stamp.nsecs'][first_index]-df_color['header.stamp.nsecs'][i])
       min_index =  rows['header.seq'][first_index]
       if len(rows) > 1:
          for j in range(first_index+1,first_index+len(rows)):
             newdiff = abs(rows['header.stamp.nsecs'][j]-df_color['header.stamp.nsecs'][i])
             if (newdiff < min_nanosec_diff):
               min_nanosec_diff =  newdiff
               min_index =  rows['header.seq'][j]
       df_color['d435i_gyro_index'][i]=min_index

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

In [21]:
#df_color
df_color.to_csv('/content/drive/MyDrive/sensorfeeds-forklift-driving-into-Pedestrian/D435I-color-indices3.csv')
